# Random Scan Gibbs Sampling Algorithm

The Random Scan Gibbs Sampling Algorithm updates only dimension in the representation of the state, chosen at random, at each update using the current values of the other dimensions of the state like this:


### $$ \mbox{Draw } j \mbox{ from a distribution on } \{1, . . . , d\} \\ X_{j}^{(t)} \sim \pi_{X_{j}|X_{-j}}(\cdot|X_{1}^{(t)}, \dots , X_{j-1}^{(t)}, X_{j+1}^{(t-1)},\dots X_{d}^{(t-1)})$$

In this notebook we will show some of implementations (good and bad examles) of the Random Scan Gibbs Sampling algorithm 

In [1]:
# Import required libraries
import seqgibbs

import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

## A good example

Let $X := (X_1,X_2) \sim N(\mu, \Sigma)$ where $\mu = (\mu_1, \mu_2)$ and covariance matrix 
$
\Sigma = \begin{pmatrix}
\sigma^2_1 & \rho \\
\rho & \sigma^2_2
\end{pmatrix}
$
.



The Gibbs sampler proceeds as follows in this case with the following conditional probabilities:

(a) Sample $X_1^{(t)} \sim N (\mu_1 +\rho/\sigma_2^2(X_2^{(t−1)}−\mu_2), \sigma_1^2−\rho^2/\sigma_2^2)$

(b) Sample $X_2^{(t)} \sim N (\mu_2 +\rho/\sigma_1^2(X_1^{(t)}−\mu_1), \sigma_2^2−\rho^2/\sigma_1^2)$.

Firstly, we take the case when the dimesnions of the target distribution are highly correlated, i.e. $\rho = 0.99$.

In [2]:
# Set correlation coefficient
rho = 0.99

# Set other parameters of the target distribution
mu1, mu2, var1, var2 = (5, 5 , 2, 1)

# Create now functions that return the parameters for the unidimensional simulator
# for X1 and X2 in terms of the current position.
# The sampler has a normal distribution shape so these parameters need
# to be the mean and variance of the dsitribution.
def first_fun(x):
    return mu1 + rho/var2 *(x - mu2), var1 - rho**2/var2

def second_fun(x):
    return mu2 + rho/var1 *(x - mu1), var2 - rho**2/var1

# Now create the unidimensional samplers for each of the two dimensions X1 and X2
first_sampler = seqgibbs.OneDimSampler(scipy.stats.norm.rvs, first_fun)
second_sampler = seqgibbs.OneDimSampler(scipy.stats.norm.rvs, second_fun)

# Now create wrapper Random Scan Gibbs sampler starting at the default position (origin)
# and to which we then feed the two unidimensional samplers we constructed.
rand_sampler = seqgibbs.RandGibbsAlgo(num_dim=2, initial_state=np.array([0, 0]))

rand_sampler.add_1_d_sampler(first_sampler)
rand_sampler.add_1_d_sampler(second_sampler)

# Run 200 complete scan cycles of the algorithm and save the chain
chain1 = rand_sampler.run(num_cycles=200)